In [91]:
import random 
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input, LSTM, TimeDistributed, Reshape
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error
import h5py

Declare constants

In [92]:
TRAIN_TEST_CUTOFF = '2020-01-31'
TRAIN_VALID_RATIO = 0.75

Metric functions for calculating F-measure

In [93]:
def _recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
 
def _precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
 
def _f1(y_true, y_pred):
    precision = _precision(y_true, y_pred)
    recall = _recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
 
def f1macro(y_true, y_pred):
    f_pos = _f1(y_true, y_pred)
    # negative version of the data and prediction
    f_neg = _f1(1-y_true, 1-K.clip(y_pred,0,1))
    return (f_pos + f_neg)/2

Model definition

In [94]:
def cnnpred_2d(seq_len=60, n_features=74, n_filters=(8,8,8), droprate=0.1):
    "2D-CNNpred model according to the paper"
    
    # model = Sequential([
    #     Input(shape=(seq_len, n_features, 1)),
    #     Conv2D(n_filters[0], kernel_size=(1, n_features), activation="relu"),
    #     Conv2D(n_filters[1], kernel_size=(3,1), activation="relu"),
    #     MaxPool2D(pool_size=(2,1)),
    #     Conv2D(n_filters[2], kernel_size=(3,1), activation="relu"),
    #     MaxPool2D(pool_size=(2,1)),
    #     Flatten(),
    #     Dropout(droprate),
    #     Dense(1, activation="sigmoid")
    # ])

    model = Sequential([
        Input(shape=(seq_len, n_features, 1)),
        Conv2D(n_filters[0], kernel_size=(1, n_features), activation="relu"),
        Reshape((seq_len, n_features)),
        LSTM(512),
        LSTM(128),
        LSTM(64),
        TimeDistributed(Dense(512)),
        Dropout(0.2),
        TimeDistributed(Dense(256)),
        Dropout(0.2),
        TimeDistributed(Dense(128)),
        TimeDistributed(Dense(64)),
        TimeDistributed(Dense(32)),
        TimeDistributed(Dense(16)),
        Dense(1)
    ])

    return model

In [95]:
def datagen(df, seq_len, batch_size, target_col, kind):
    """A generator to produce samples for Keras model"""

    batch = []

    while True:

        # Set up splitting parameters
        input_cols = [c for c in df.columns if c != target_col]
        index = df.index[df.index < TRAIN_TEST_CUTOFF]
        split = int(len(index) * TRAIN_VALID_RATIO)

        # Range for the training set
        if kind == 'train':
            index = index[:split]

        # Range for the validation set
        elif kind == 'valid':
            index = index[split:]   

        while True:
            "Pick one position, then clip a sequence length"

            # Pick one time step
            t = random.choice(index)

            # Find its position in the DataFrame      
            n = (df.index == t).argmax()

            # Start over if there isn't enough data for one sequence length  
            if (n - seq_len + 1) < 0:
                continue
            
            # Create the DataFrame of one sequence length
            frame = df.iloc[n - seq_len+1 : n+1]

            # Append X and y values as a sample in the CNN dataset
            batch.append([frame[input_cols].values, df.loc[t, target_col]])

            break

        # If we get enough for a batch, yield the instance
        if len(batch) == batch_size:

            # Unpack the `batch` list into features and target
            X, y = zip(*batch)

            # Expand dimensions of X
            X, y = np.expand_dims(np.array(X), 3), np.array(y)

            # Yield the sample
            yield X, y

            # Clear the batch list for next iteration
            batch = []

In [96]:
def testgen(df, seq_len, target_col):
    "Return array of all test samples"

    batch = []

    input_cols = [c for c in df.columns if c != target_col]

    # find the start of test sample
    t = df.index[df.index > TRAIN_TEST_CUTOFF][0]
    n = (df.index == t).argmax()

    for i in range(n+1, len(df)+1):

        frame = df.iloc[i-seq_len:i]
        batch.append([frame[input_cols].values, frame[target_col][-1]])

    X, y = zip(*batch)

    return np.expand_dims(np.array(X),3), np.array(y)

In [97]:
data = pd.read_csv('../csv/initial_variables.csv', index_col='date', parse_dates=True, infer_datetime_format=True)

In [98]:
cols = data.columns

# If the current price is higher than yesterday's price then target = 1, else 0
data['target'] = (data['close'].pct_change().shift(-1) > 0).astype(int)

data.dropna(inplace=True)

# Fit the standard scaler using the training dataset
index = data.index[data.index > TRAIN_TEST_CUTOFF]
index = index[:int(len(index) * TRAIN_VALID_RATIO)]
scaler = StandardScaler().fit(data.loc[:, cols])

# Save scale transformed dataframe
data[cols] = scaler.transform(data[cols])
# data

In [99]:
seq_len = 60
batch_size = 128
n_epochs = 20
n_features = 74
 
model = cnnpred_2d(seq_len, n_features)
model.compile(optimizer='adam', loss='mae', metrics=['acc', f1macro])
model.summary()

ValueError: Exception encountered when calling layer "reshape_7" (type Reshape).

total size of new array must be unchanged, input_shape = [60, 1, 8], output_shape = [60, 74]

Call arguments received by layer "reshape_7" (type Reshape):
  • inputs=tf.Tensor(shape=(None, 60, 1, 8), dtype=float32)

In [ ]:
checkpoint_path = "./models/cp2d-{epoch}-{val_f1macro:.2f}.h5"

callbacks = [
    ModelCheckpoint(
        checkpoint_path,
        monitor='val_f1macro', 
        mode="max", verbose=0,
        save_best_only=True, 
        save_weights_only=False, 
        save_freq="epoch"
    )
]

In [ ]:
training_gen = datagen(data, seq_len, batch_size, 'target', 'train')
validation_gen = datagen(data, seq_len, batch_size, 'target', 'valid')

model.fit(
    training_gen,
    validation_data=validation_gen,
    epochs=n_epochs, 
    steps_per_epoch=400, 
    validation_steps=10, 
    verbose=1,
    callbacks=callbacks
)

Epoch 1/20
400/400 [==============================] - 30s 76ms/step - loss: 0.4506 - acc: 0.5494 - f1macro: 0.3540 - val_loss: 0.4633 - val_acc: 0.5367 - val_f1macro: 0.3485
Epoch 2/20
400/400 [==============================] - 31s 77ms/step - loss: 0.4567 - acc: 0.5433 - f1macro: 0.3515 - val_loss: 0.4406 - val_acc: 0.5594 - val_f1macro: 0.3583
Epoch 3/20
400/400 [==============================] - 31s 77ms/step - loss: 0.4505 - acc: 0.5495 - f1macro: 0.3541 - val_loss: 0.4477 - val_acc: 0.5523 - val_f1macro: 0.3554
Epoch 4/20
400/400 [==============================] - 30s 76ms/step - loss: 0.4539 - acc: 0.5461 - f1macro: 0.3527 - val_loss: 0.4250 - val_acc: 0.5750 - val_f1macro: 0.3644
Epoch 5/20
400/400 [==============================] - 30s 75ms/step - loss: 0.4564 - acc: 0.5436 - f1macro: 0.3516 - val_loss: 0.4406 - val_acc: 0.5594 - val_f1macro: 0.3584
Epoch 6/20
400/400 [==============================] - 31s 77ms/step - loss: 0.4560 - acc: 0.5440 - f1macro: 0.3518 - val_loss: 0.4

In [ ]:
# Prepare test data
test_data, test_target = testgen(data, seq_len, 'target')

In [ ]:
# Test the model
test_out = model.predict(test_data)
test_pred = (test_out > 0.5).astype(int)
print('accuracy:', accuracy_score(test_pred, test_target))
print('MAE:', mean_absolute_error(test_pred, test_target))
print('F1:', f1_score(test_pred, test_target))

20/20 [==============================] - 0s 3ms/step
accuracy: 0.5421303656597775
MAE: 0.4578696343402226
F1: 0.7030927835051547


In [ ]:
model.save('./models/final_CNN.h5')